### setting up the environment the first time

Complications always arise when setting this up for the first time. So I'm going to *attempt* to standardize this somewhat for me.

Current config: Windows10pro but running on WSL2

## Need to install an X11 graphical monitor on windows. 
Interents recommented [VcXsrv](https://sourceforge.net/projects/vcxsrv/) [Video Instruction](https://www.youtube.com/watch?v=4SZXbl9KVsw) [full writeup](https://www.gregbrisebois.com/posts/chromedriver-in-wsl2/)

Also have to update firewall permission to allow VcXsrv

.
## to install the google chrome webdiver ##

Step 1 – Add Google Chrome PPA

First, add (if not added already) the Google Chrome repository on your system using the following command. While using PPA to our system we also receive the latest updates whenever you check for system updates.

<code> wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | sudo apt-key add - </code>

Next, create a PPA file for Google chrome on your system by running command:

<code> sudo sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google.list' </code>

Step 2 – Install or Upgrade Google Chrome

After adding the Google Chrome repository in our system use following commands to install the latest Google Chrome stable release. If you already have installed an older version, It will upgrade the currently installed version with the recent stable version.

<code> sudo apt-get update
sudo apt-get install google-chrome-stable </code>

# NOTE: Cannot get selenium to run on WSL2 and a Jupyter Notebook

This notebook will be used to post-scrape analysis

In [1]:
# Data Basics
import pandas as pd
import numpy as np
from datetime import datetime as dt
pd.options.display.float_format = "{:.0f}".format



In [13]:
data = pd.read_csv("DATA/scrape_files/partial_20201116.csv")
data.tail()

,Unnamed: 0,name,address,unit,sqft,bed,bath,price,city,state,zipcode,description,details,url,date
10345,10345,1315 Norwalk Ln,1315 Norwalk Ln,Unit 206,447 sqft,1bd,1ba,Contact$950,Austin,TX,78703,FREE Rent - Move in Today! - 450 square foot o...,"Heating: Forced Air, Electric, Gas ,Days on Ma...",https://www.trulia.com/c/tx/austin/1-bed-544-s...,2020-11-16
10346,10346,"1 bed, $819",909 W 26th St,1 Bed 1.0 Bath,NaN,1bd,1ba,$819,Austin,TX,78705,"OH, WHAT A DEAL! STUDIO 1-1 APARTMENT, COGNAC ...","Property Type: Apartment ,Parking: None ,Cats ...",https://www.trulia.com/c/tx/austin/1-bed-829-9...,2020-11-16
10347,10347,"1 bed, $819",909 W 26th St,Unit 105,NaN,1bd,1ba,Contact$819,Austin,TX,78705,"OH, WHAT A DEAL! STUDIO 1-1 APARTMENT, COGNAC ...","Property Type: Apartment ,Parking: None ,Cats ...",https://www.trulia.com/c/tx/austin/1-bed-829-9...,2020-11-16
10348,10348,"2 bed, $1,119",3205 Helms St,2 Bed 1.0 Bath,"3,596 sqft",2bd,1ba,"$1,119",Austin,TX,78705,"OH, WHAT A DEAL! UT AREA 2-1 APARTMENT, VINTAG...","Days on Market: 20 Days on Trulia ,Property Ty...",https://www.trulia.com/c/tx/austin/2-bed-1-119...,2020-11-16
10349,10349,"2 bed, $1,119",3205 Helms St,Unit 201,"3,596 sqft",2bd,1ba,"Contact$1,119",Austin,TX,78705,"OH, WHAT A DEAL! UT AREA 2-1 APARTMENT, VINTAG...","Days on Market: 20 Days on Trulia ,Property Ty...",https://www.trulia.com/c/tx/austin/2-bed-1-119...,2020-11-16


In [12]:
data.isna().sum()

Unnamed: 0       0
name             0
address          0
unit           228
sqft            59
bed              0
bath             0
price            0
city             0
state            0
zipcode          0
description      0
details          0
url              0
date             0
dtype: int64

256

1
2
3
4
5
6
7
8
9


In [18]:
cities = [
    ["Chicago", "IL"],
    ["Saint_Louis", "MO"],
    ["New_York", "NY"],
    ["Las_Vegas", "NV"],
    ["Dallas", "TX"],
    ["Portland", "OR"],
    ["Seattle", "WA"],
    ["Minneapolis", "MN"],
    ["Orlando", "FL"],
    ["San_Francisco","CA"]
]

In [17]:
for i, city in enumerate(cities, start=1):
    print(i, city[0])

1 Chicago
2 Saint_Louis
3 New_York
4 Las_Vegas
5 Dallas
6 Portland
7 Seattle
8 Minneapolis
9 Orlando


In [20]:
dt.today().strftime('%Y%m%d')

'20201116'

""" Some regex notes
Everthing before a comma
<code> ^(.+?), </code>

Assuming the date is always in the format: [MM]-[DD]-[YYYY].
([0-9]{2}\-[0-9]{2}\-[0-9]{4})

How I do timestamps in filenames
([0-9]{2}[0-9]{2}[0-9]{4})

"""

In [28]:
import os, re 

today = int(dt.today().strftime('%Y%m%d'))

for i in range(7):
    if os.path.isfile(f"DATA/scrape_files/partial_{today - i}.csv"):
        data = pd.read_csv(f"DATA/scrape_files/partial_{today - i}.csv")
        break # fyi, only breaks one-level out of for-loop
    elif i < 7:
        print("waiting...")
    else:
        print("No recent file found, generating new list")
        

0
waiting...
1
waiting...
2
waiting...
3
waiting...


In [42]:
from tqdm import tqdm

base_url = "https://www.trulia.com"

cities = [
    
    ["Saint_Louis", "MO"],
    ["Chicago", "IL"],
    ["New_York", "NY"],
    ["Las_Vegas", "NV"],
    ["Dallas", "TX"],
    ["Portland", "OR"],
    ["Seattle", "WA"],
    ["Minneapolis", "MN"],
    ["Orlando", "FL"],
    ["San_Francisco", "CA"],
    ["Austin", "TX"],
    ["Ann_Arbor", "MI"],
]

for i, city_state in enumerate(tqdm(cities, unit="city"), start=1):
    today = int(dt.today().strftime("%Y%m%d"))
    city, state = city_state
    print(city, state)
    city_url = f"/for_rent/{city},{state}/"
    residence_urls = f"DATA/urls/apt_page_listings_{city}_{state}_{today}.csv"
    unit_info = f"DATA/scrape_files/apt_unit_listings_{city}_{state}_{today}.csv"

    # Generate list of URLs to walk through, skip if saved list is recent
    for i in range(7):
        if os.path.isfile(
            f"DATA/urls/apt_page_listings_{city}_{state}_{today - i}.csv"
        ):
            url_list = pd.read_csv(
                f"DATA/urls/apt_page_listings_{city}_{state}_{today - i}.csv"
            )
            print("Blah")
            break  # only breaks one-level out of for-loop
        elif i < 6:
            print("nah")
            continue
        else:
            print("No recent file found, generating new list")
            # ulist = bot.get_url_list(base_url, city_url)
            # to_save = pd.DataFrame(ulist)
            # to_save.to_csv(residence_urls)
            # url_list = pd.read_csv(residence_urls)


    # Find all the units available for a listing
    if os.path.isfile(unit_info):
        print("units file found")
    else:
        print("units file not found")
        # apts_data = bot.get_all_apartments(base_url, url_list, city, state)
        # to_save = pd.DataFrame(apts_data)
        # to_save.to_csv(unit_info)

100%|██████████| 12/12 [00:00<00:00, 324.59city/s]Saint_Louis MO
Blah
units file found
Chicago IL
Blah
units file found
New_York NY
Blah
units file not found
Las_Vegas NV
nah
nah
nah
nah
nah
nah
No recent file found, generating new list
units file not found
Dallas TX
nah
nah
nah
nah
nah
nah
No recent file found, generating new list
units file not found
Portland OR
nah
nah
nah
nah
nah
nah
No recent file found, generating new list
units file not found
Seattle WA
nah
nah
nah
nah
nah
nah
No recent file found, generating new list
units file not found
Minneapolis MN
nah
nah
nah
nah
nah
nah
No recent file found, generating new list
units file not found
Orlando FL
nah
nah
nah
nah
nah
nah
No recent file found, generating new list
units file not found
San_Francisco CA
nah
nah
nah
nah
nah
nah
No recent file found, generating new list
units file not found
Austin TX
nah
nah
nah
nah
nah
nah
No recent file found, generating new list
units file not found
Ann_Arbor MI
nah
nah
nah
nah
nah
nah
No recent 

In [ ]:
home_deets = soup.find_all("div", {"data-testid": "home-details-summary-container"})

price = home_deets[0].find_all("div", "Text__TextBase-sc-1i9uasc-0-div Text__TextContainerBase-sc-1i9uasc-1 qAaUO")[0].text

name,
***address,
unit,
sqft,
bed,
bath,
price,
city,
state,
zipcode,
description,
details,
"url": apartment_url,
"date": date,

# good to go
price = home_deets[0].find_all("div", lambda L: L and L.startswith("Text__TextBase"))[0].text
bed = home_deets[0].find_all("div", lambda L: L and L.startswith("MediaBlock__MediaContent"))[0].text
bath = home_deets[0].find_all("div", lambda L: L and L.startswith("MediaBlock__MediaContent"))[1].text
sqft = home_deets[0].find_all("div", lambda L: L and L.startswith("MediaBlock__MediaContent"))[2].text
name = home_deets[0].find_all("span", {"data-testid": "home-details-summary-headline"})[0].text
city_state_zip = home_deets[0].find_all("span", {"data-testid": "home-details-summary-city-state"})[0].text
description = soup.find_all("div", {"data-testid": "home-description-text-description-text"})[0].text
details = [detail.text for detail in soup.find_all("li", {"class": lambda L: L and L.startswith("FeatureList__FeatureListItem")})]
unit = "home"
date = str(dt.now().date())
apartment_url = base_url + current_url


In [47]:
blah = []

if blah != []:
    print("yo")
else:
    print("nah")

nah
